In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import opttrot


## Fast Pauli decomposition algorithm implementation

```
Hantzko, Binkowski, and Gupta, Tensorized Pauli decomposition algorithm
```

In [3]:
import numpy as np
from copy import deepcopy

In [4]:
from opttrot import utils 

In [5]:
# Size = 2^n Hermit matrix
n =5
size_n = int(2**n)
A = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
ImA = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
A = A.H@A
ImA = ImA.H@ImA
H = A+1j*ImA

In [6]:
pauli_matrix, pauli_terms = opttrot.Hamiltonian.generate_pauli_terms(n) # Get 2^n pauli-terms

In [7]:
# Frobenius inner product
decomposition = {}
for matrix, term in zip(pauli_matrix, pauli_terms):
    coef = utils.frobenius_inner(matrix, H)
    decomposition[term] = coef

In [8]:
decomposition

{'IIIII': (10.94809950633871+10.546187698643635j),
 'IIIIZ': (-0.40897761418312095-0.2533647851837326j),
 'IIIZI': (0.08174090598881989-0.06425085050572932j),
 'IIIZZ': (-0.4423449882487067+0.1806186898684512j),
 'IIZII': (-0.28838401483963444+0.7747920073594379j),
 'IIZIZ': (-0.16779181073749633+0.11109102720467359j),
 'IIZZI': (0.14546227363878766-0.17250685701980206j),
 'IIZZZ': (-0.24144858722482615+0.23339870751192518j),
 'IZIII': (-0.009930849354064575+0.07977878917211556j),
 'IZIIZ': (-0.06490834815075458+0.5198175000556363j),
 'IZIZI': (-0.3875188230434721+0.3805192184620598j),
 'IZIZZ': (-0.5985184591114323+0.10421652845589102j),
 'IZZII': (0.08947989826077529-0.2693831412381241j),
 'IZZIZ': (0.7272569347944577-0.09643512819370065j),
 'IZZZI': (0.005233299216528375-0.26709657839531153j),
 'IZZZZ': (-0.15566115003220538+0.19425008742237249j),
 'ZIIII': (-0.24733249796285+0.2647530627432653j),
 'ZIIIZ': (-0.008093976982675166+0.05445194635752515j),
 'ZIIZI': (0.1848900308624593+

In [9]:
H_test = deepcopy(H)

### Implementation

In [10]:
H_test = deepcopy(H)
exp2n = H_test.shape[0]
n = int(np.log2(exp2n))

print("Hamiltonian shape:", H_test.shape)
print("Total qubits:", n)


Hamiltonian shape: (32, 32)
Total qubits: 5


In [11]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5


In [12]:
def tensorized_decomposition(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n)) # wires

    l = int(exp2n)

    for i in range(n): #range(n)
        m = int(2**i) # Number of submatrix
        l = int(l/2) # Sub matrix size, square
        for j in range(m):
            for k in range(m):
                num_i = j*(2*l) # Initial position of sub matrix row
                num_j = k*(2*l) # Initial position of sub matrix column
                #print(f"{num_i}, {num_j}")
                #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")

                H[num_i: num_i+l, num_j:num_j+l]        += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
        H *= 0.5
    return H

In [37]:
tensorized_decomposition(pstr_to_matrix("ZXY"))

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j]])

In [13]:
def index_xzcode(i, j):
    return i^j, i
def xzcode_index(nx, nz):
    return nz, nx^nz

In [14]:
opttrot.utils.pstr_from_xz(2, 4)

'ZXI'

In [15]:
H_test = deepcopy(H)

In [16]:
H_decom = tensorized_decomposition(H_test)
nr, nc = H_decom.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_decom[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [17]:
nr, nc = H_test.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_test[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [18]:
decom

{'IIIII': (10.94809950633871+10.546187698643637j),
 'IIIIX': (8.192545291503944+8.205107618682444j),
 'IIIXI': (8.246220361326401+8.01165803837808j),
 'IIIXX': (8.267871990078516+7.882540141713246j),
 'IIXII': (8.22881312208163+7.835500709726022j),
 'IIXIX': (8.294776376241497+7.798048459650887j),
 'IIXXI': (8.317742804811125+7.864020848629601j),
 'IIXXX': (8.24836391603958+7.767882892070036j),
 'IXIII': (8.602501577043324+7.9337939559601125j),
 'IXIIX': (8.269585635326763+8.006116029551865j),
 'IXIXI': (8.44714133417422+7.828529078932311j),
 'IXIXX': (8.264732846803694+8.195071269016797j),
 'IXXII': (8.21064223773745+7.748282378976177j),
 'IXXIX': (8.115577048451986+7.774997457774438j),
 'IXXXI': (8.235241578541245+7.885200099196872j),
 'IXXXX': (8.43687246325831+7.903529856177475j),
 'XIIII': (8.366464484249999+7.839058276913546j),
 'XIIIX': (8.459218162641832+7.991596256972912j),
 'XIIXI': (8.373054529889274+8.193171605916802j),
 'XIIXX': (8.11706476636478+7.886053162060575j),
 'XIX

In [19]:
diff = {}
for key in decom.keys():
    diff[key] = abs(decomposition[key] - decom[key])

In [20]:
np.array(list(diff.values())).max()

1.9860273225978185e-15

In [21]:
from pd import PauliDecomposition

In [22]:
pstring, pcoef = PauliDecomposition(H)
pdecompose = {st:coef for st, coef in zip(pstring, pcoef)}

In [23]:
pdecompose

{'IIIII': (10.94809950633871+10.546187698643637j),
 'IIIIX': (8.192545291503944+8.205107618682444j),
 'IIIIZ': (-0.4089776141831214-0.2533647851837326j),
 'IIIXI': (8.246220361326401+8.01165803837808j),
 'IIIXX': (8.267871990078516+7.882540141713246j),
 'IIIXZ': (-0.5716242848644062-0.1458974923800911j),
 'IIIYY': (0.3304114795699622-0.16382343983947045j),
 'IIIZI': (0.08174090598881989-0.0642508505057302j),
 'IIIZX': (0.10664499620201884+0.02700571051968126j),
 'IIIZZ': (-0.4423449882487063+0.1806186898684512j),
 'IIXII': (8.22881312208163+7.835500709726022j),
 'IIXIX': (8.294776376241497+7.798048459650887j),
 'IIXIZ': (-0.3698470438368062-0.3878580649803016j),
 'IIXXI': (8.317742804811125+7.864020848629601j),
 'IIXXX': (8.24836391603958+7.767882892070036j),
 'IIXXZ': (-0.18030393908994125-0.30608564353679224j),
 'IIXYY': (0.37396572427127506-0.24455227607535512j),
 'IIXZI': (-0.1566715714452276-0.1789169117648166j),
 'IIXZX': (0.13450059362074018-0.08240935196873433j),
 'IIXZZ': (-0.

In [24]:
diff2 = {}
for key in decom.keys():
    try:
        diff2[key] = abs(pdecompose[key] - decom[key])
    except:
        print(decom[key])

0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
-4.440892098500626e-16j
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
4.440892098500626e-16j
0j
8.881784197001252e-16j
0j
0j
0j
4.440892098500626e-16j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
(4.440892098500626e-16-2.220446049250313e-16j)
-6.661338147750939e-16j
(-2.220446049250313e-16+0j)
0j
(4.440892098500626e-16-4.440892098500626e-16j)
(-4.440892098500626e-16-4.440892098500626e-16j)
0j
0j
(6.661338147750939e-16-2.220446049250313e-16j)
(4.440892098500626e-16+8.881784197001252e-16j)
(2.220446049250313e-16-2.220446049250313e-16j)
0j
(-4.440892098500626e-16+0j)
(-2.220446049250313e-16+0j)
(-2.220446049250313e-16-4.440892098500626e-16j)
0j
-2.220446049250313e-16j
0j
0j
0j
(4.440892098500626e-16+4.440892098500626e-16j)
0j
0j
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
(6.661338147750939e-16+0j)
0j
0j
0j
(-4.440892098500626e-16+2.220446049250313e-16j)
0j
0j
0j
-4.440892098500626e-16j
0j
0j
0j
(-2.220446049250313e-16+2.220446049250313

In [14]:
def fast_p_decompose(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n))
    # Implementation of decomposition
    l = int(exp2n)
    for i in range(n):
        m = int(2**i)
        l = int(l/2)
        for j in range(m):
            for k in range(m):

                num_i = j*l
                num_j = k*l

                H[num_i: num_i+l, num_j:num_j+l] += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j

        H *= 0.5
    return H

# string to matrix

In [26]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from opttrot.utils import pstr_to_matrix, pstr_to_xz_fam_code, xz_fam_code_to_pstr, xzcode_index, tp_decomposition

In [27]:
pstr= "ZX"
xz_code = pstr_to_xz_fam_code(pstr)
p_matrix = pstr_to_matrix(pstr)

In [28]:
p_matrix

array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j]])

In [29]:
xz_code # IX = 01, ZI=10

(1, 2)

In [30]:
mat = np.zeros(p_matrix.shape)
mat[*xzcode_index(*xz_code)] = 1

In [31]:
mat

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [32]:
from copy import deepcopy

In [33]:
mat

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [34]:
tp_decomposition(p_matrix)

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
       [-0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [35]:
# 0, 1
# y = (x+1)/2
# y+y-1

In [36]:
# Routines
i,j = p_index = xzcode_index(*xz_code)
print("P index:", p_index)
print("1) I-Z/X-Y:", "X-Y" if (i+j)%2 else "I-Z")
print("2) (1/2, 1/2):", (i%2, j%2))
print("---", "Ori:", (i, j), "Cor:", (i+1-2*(i%2), j+ 1-(2*(j%2))))
pair = ((i, j), (i+1-2*(i%2), j+ 1-(2*(j%2))))

l = 1
print("Size:", l)
print(f"Range:[{l*pair[0][0]}: {l*pair[0][0] +l}, {l*pair[0][1]}:{l*pair[0][1]+l}], [{l*pair[1][0]}: {l*pair[1][0] +l}, {l*pair[1][1]}:{l*pair[1][1]+l}]")
print("New pair:", (i>>1, j>>1)) # Next pair

P index: (2, 3)
1) I-Z/X-Y: X-Y
2) (1/2, 1/2): (0, 1)
--- Ori: (2, 3) Cor: (3, 2)
Size: 1
Range:[2: 3, 3:4], [3: 4, 2:3]
New pair: (1, 1)


In [37]:
mat_copy = deepcopy(mat)

In [38]:
A12 = mat[2:3, 3:4] - 1j*mat[3:4, 2:3]
A21 = mat[2:3, 3:4] + 1j*mat[3:4, 2:3]

### Implementation

From pauli polynomial to Hamiltonian

In [43]:
#P-xz polynomial-------
qubits = 3
pxzcodes = [
    (3,   (1, 4)),
    (9.2, (2, 3)),
    (2.1, (6, 3))
]
#matrix----------------
mat = np.zeros((2**qubits, 2**qubits), dtype=complex)
#P-index--------------
p_indexes = [xzcode_index(*xz_code) for (c, xz_code) in pxzcodes]
for pxz in pxzcodes:
    i, j = xzcode_index(*pxz[1])
    mat[i, j] = pxz[0]
#---------------------


In [ ]:
index_to_xzcode(9, 14)

In [44]:
p_indexes

[(4, 5), (3, 1), (3, 5)]

In [ ]:

steps = int(np.log2(mat.shape[0]))
unit_size= 1

in_ps = [p for p in p_indexes]
for step in range(steps):
    #print("========================================")
    psteps =[]
    dup = []
    #----
    for p in in_ps:
        i, j = p
        p_class = (i+j)%2 #0: IZ, 1:XY
        n, o = i%2, j%2 # (1), (2) determination
        l, m = (i+1-2*(n), j+ 1-(2*(o))) # get a corresponding location

        if (l,m) in dup: # Eliminate duplicated operation.
            continue
        elif (i, j) in dup:
            dup.append((l,m))
            continue
        else:
            dup.append((i,j))
            dup.append((l,m))
             
        if n: # (2)
            pair = ((l, m), (i, j))
        else: #(1)
            pair = ((i, j), (l, m))
        
        #print((i,j), (l,m))
        #print(dup)
        #print("XY" if p_class else "IZ")
        #print("(2)" if n else "(1)")
        #print("size:", unit_size)
        #print(f"{unit_size*pair[0][0]}:{unit_size*pair[0][0]+unit_size},  {unit_size*pair[0][1]}:{unit_size*pair[0][1]+unit_size}")
        #print(f"{unit_size*pair[1][0]}: {unit_size*pair[1][0]+unit_size}, {unit_size*pair[1][1]}:{unit_size*pair[1][1]+unit_size}")
        
        coef = -1j if p_class else 1 # ture: XY, false: IZ
       
        mat[unit_size*pair[0][0]: unit_size*pair[0][0]+unit_size, unit_size*pair[0][1]:unit_size*pair[0][1]+unit_size] += coef*mat[unit_size*pair[1][0]: unit_size*pair[1][0] +unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size]
        mat[unit_size*pair[1][0]: unit_size*pair[1][0]+unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size] =       mat[unit_size*pair[0][0]: unit_size*pair[0][0] +unit_size, unit_size*pair[0][1]:unit_size*pair[0][1]+unit_size] -2*coef *mat[unit_size*pair[1][0]: unit_size*pair[1][0] +unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size]
        
        i >>=1
        j >>=1
        if (i, j) in psteps:
            continue
        else:
            psteps.append((i,j))
    #----
    in_ps = [p for p in psteps]
    unit_size *=2
    #print(in_ps)
    #print(mat)

In [40]:
mat

array([[ 0.+0.j ,  3.+0.j ,  0.-9.2j,  0.+0.j ,  0.+0.j ,  0.+0.j ,
         0.-2.1j,  0.+0.j ],
       [ 3.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+2.1j],
       [ 0.+9.2j,  0.+0.j ,  0.+0.j ,  3.+0.j ,  0.+2.1j,  0.+0.j ,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.-9.2j,  3.+0.j ,  0.+0.j ,  0.+0.j ,  0.-2.1j,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j , -3.+0.j ,
         0.-9.2j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+2.1j, -3.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+9.2j],
       [ 0.+2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,
         0.+0.j , -3.+0.j ],
       [ 0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.-9.2j,
        -3.+0.j ,  0.+0.j ]])

In [41]:
# p_xz codes to matrix
pxzcodes
mat_ref = np.zeros(mat.shape, dtype=complex)
for pxz in pxzcodes:
    coef, xz = pxz
    pstr = xz_fam_code_to_pstr(xz, 3)
    pmat = pstr_to_matrix(pstr)
    mat_ref += coef*pmat
mat_ref

array([[ 0.+0.j ,  3.+0.j ,  0.-9.2j,  0.+0.j ,  0.+0.j ,  0.+0.j ,
         0.-2.1j,  0.+0.j ],
       [ 3.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+2.1j],
       [ 0.+9.2j,  0.+0.j ,  0.+0.j ,  3.+0.j ,  0.+2.1j,  0.+0.j ,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.-9.2j,  3.+0.j ,  0.+0.j ,  0.+0.j ,  0.-2.1j,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j , -3.+0.j ,
         0.-9.2j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+2.1j, -3.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+9.2j],
       [ 0.+2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,
         0.+0.j , -3.+0.j ],
       [ 0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.-9.2j,
        -3.+0.j ,  0.+0.j ]])

In [42]:
# Single Pauli term to matrix

In [ ]:
# Multiple Pauli term at once
# Inverse routine of decomposition

In [ ]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5
